In [1]:
#!pip install -U openpyxl

In [2]:
import os
import pandas as pd
import numpy as np
from datetime import date, datetime
from openpyxl import load_workbook
import warnings
warnings.filterwarnings('ignore')

In [3]:
dh_inicio = datetime.today()

### CARGA DO ARQUIVO DA EVOLUÇÃO MASTER

In [4]:
# Caminho da pasta dos arquivos
df_master_evolucao = pd.read_excel(r"C:\Projetos\PPSI\BASE\MASTER-Evolucao-C1C2C3C4.xlsx" , sheet_name = "Evolucao-PPSI")

In [5]:
# Renomeando a coluna
df_master_evolucao = df_master_evolucao.rename(columns={'CICLO': 'Ciclo_Entrega'})

In [6]:
df_master_evolucao = df_master_evolucao.rename(columns={'Estrut. Básica': 'iBase'})

In [7]:
df_master_evolucao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Categoria      1014 non-null   object
 1   Órgão          1014 non-null   object
 2   iBase          1010 non-null   object
 3   iSeg           1010 non-null   object
 4   iPriv          1010 non-null   object
 5   Ciclo_Entrega  1014 non-null   int64 
 6   ANO            1014 non-null   int64 
 7   SEMESTRE       1014 non-null   int64 
 8   Região         1014 non-null   object
dtypes: int64(3), object(6)
memory usage: 71.4+ KB


In [8]:
df_master_evolucao.head()

,Categoria,Órgão,iBase,iSeg,iPriv,Ciclo_Entrega,ANO,SEMESTRE,Região
0,Agência,ABIN,1,0.73,0.52,1,2023,1,Centro-Oeste
1,Agência,AEB,0.9,0.76,0.41,1,2023,1,Centro-Oeste
2,Especiais,AGU,0.26,0.39,0.23,1,2023,1,Centro-Oeste
3,Agência,ANA,1,0.72,0.88,1,2023,1,Centro-Oeste
4,Agência,ANAC,1,0.546,0.67,1,2023,1,Centro-Oeste


In [9]:
# Renomeando os valores da coluna `ciclo_entrega`
df_master_evolucao['Ciclo_Entrega'] = df_master_evolucao['Ciclo_Entrega'].apply(lambda x: f'C{x}')

In [10]:
df_master_evolucao.head()

,Categoria,Órgão,iBase,iSeg,iPriv,Ciclo_Entrega,ANO,SEMESTRE,Região
0,Agência,ABIN,1,0.73,0.52,C1,2023,1,Centro-Oeste
1,Agência,AEB,0.9,0.76,0.41,C1,2023,1,Centro-Oeste
2,Especiais,AGU,0.26,0.39,0.23,C1,2023,1,Centro-Oeste
3,Agência,ANA,1,0.72,0.88,C1,2023,1,Centro-Oeste
4,Agência,ANAC,1,0.546,0.67,C1,2023,1,Centro-Oeste


In [11]:
# Manter a coluna como float
df_master_evolucao['iBase'] = pd.to_numeric(df_master_evolucao['iBase'], errors='coerce')

# Exibir com vírgula (sem alterar tipo para 'object')
df_master_evolucao['iBase'] = df_master_evolucao['iBase'].apply(lambda x: f'{x:.2f}'.replace('.', ',') if pd.notnull(x) else '')


# Manter a coluna como float
df_master_evolucao['iSeg'] = pd.to_numeric(df_master_evolucao['iSeg'], errors='coerce')

# Exibir com vírgula (sem alterar tipo para 'object')
df_master_evolucao['iSeg'] = df_master_evolucao['iSeg'].apply(lambda x: f'{x:.2f}'.replace('.', ',') if pd.notnull(x) else '')

# Manter a coluna como float
df_master_evolucao['iPriv'] = pd.to_numeric(df_master_evolucao['iPriv'], errors='coerce')

# Exibir com vírgula (sem alterar tipo para 'object')
df_master_evolucao['iPriv'] = df_master_evolucao['iPriv'].apply(lambda x: f'{x:.2f}'.replace('.', ',') if pd.notnull(x) else '')



In [12]:
df_master_evolucao = df_master_evolucao.rename(columns={'Órgão': 'Nome da Origem.1'})

In [13]:
# Substitui "C1" por "Ciclo 1", "C2" por "Ciclo 2", etc.
df_master_evolucao['Ciclo_Entrega'] = df_master_evolucao['Ciclo_Entrega'].str.extract(r'(\d+)')[0].astype(int).apply(lambda x: f'Ciclo {x}')

In [14]:
# Substitui "C1" por "Ciclo 1", "C2" por "Ciclo 2", etc.
df_master_evolucao['Ciclo_Entrega'] = df_master_evolucao['Ciclo_Entrega'].str.extract(r'(\d+)')[0].astype(int).apply(lambda x: f'Ciclo {x}')
# Salva em Excel
df_master_evolucao.to_csv(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\FONTE_PAINEL\Evolucao-PPSI_C.csv", index=False)

### MASTER_INDICADORES_CONSOLIDADO

In [15]:
# Caminho do arquivo
arquivo = (r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\BASE\2025-Master - Lista Orgaos SISP - Agrupamento PPSI - Protected.xlsx")

# Mapeamento: nome da aba → (índices das colunas a ler, nomes novos)
colunas_por_aba = {
    'Historico-MASTER-C1': 
                ([1, 2, 6, 7, 8, 12, 13, 14, 15, 16, 17, 18, 19], 
                ['Categoria','Órgão', 'iBase', 'iSeg', 'iPriv', 'Tipo', 'Órgão.1', 
                 'Órgão-Sigla', 'Órgão-Nome', 'Setor', 'UF', 'Município', 'Região']), 
    'Historico-MASTER-C2':     
                ([2, 3, 7, 8, 9, 16, 17, 20, 21, 22, 23, 24, 25],  
                ['Categoria','Órgão', 'iBase', 'iSeg', 'iPriv', 'Tipo', 'Órgão.1', 
                 'Órgão-Sigla', 'Órgão-Nome', 'Setor', 'UF', 'Município', 'Região']),
    'Historico-MASTER-C3': 
                ([2, 3, 7, 8, 9, 19, 20, 23, 24, 25, 26, 27, 28],  
                ['Categoria','Órgão', 'iBase', 'iSeg', 'iPriv', 'Tipo', 'Órgão.1', 
                 'Órgão-Sigla', 'Órgão-Nome', 'Setor', 'UF', 'Município', 'Região']),
    'Atual-MASTER-C4': 
                ([3, 4, 8, 9, 10, 23, 24, 27, 28, 29, 30, 31, 32], 
                ['Categoria','Órgão', 'iBase', 'iSeg', 'iPriv', 'Tipo', 'Órgão.1', 
                 'Órgão-Sigla', 'Órgão-Nome', 'Setor', 'UF', 'Município', 'Região'])
}

# Mapeamento: nome da aba → valor da coluna 'CICLO'
ciclo_por_aba = {
    'Historico-MASTER-C1': 'C1',
    'Historico-MASTER-C2': 'C2',
    'Historico-MASTER-C3': 'C3',
    'Atual-MASTER-C4': 'C4'    
}

tabelas = []

for nome_aba, (indices, nomes_colunas) in colunas_por_aba.items():
    # Lê as colunas por índice
    df = pd.read_excel(arquivo, sheet_name=nome_aba, usecols=indices, header=0)
    
    # Renomeia as colunas conforme desejado
    df.columns = nomes_colunas
    
    # Adiciona colunas auxiliares
   # df['aba'] = nome_aba
    df['Ciclo_Entrega'] = ciclo_por_aba[nome_aba]

    # Mapas para ano e semestre
    ano_map = {1: 2023, 2: 2023, 3: 2024, 4: 2024}
    semestre_map = {1: 1, 2: 2, 3: 1, 4: 2}
    
    # Criar colunas ano e semestre
#    df['ANO'] = df['Ciclo_Entrega'].map(ano_map)
#    df['SEMESTRE'] = df['Ciclo_Entrega'].map(semestre_map)

    tabelas.append(df)

# Junta tudo
df_master_indicadores = pd.concat(tabelas, ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\denis\\OneDrive\\Documentos\\python\\meuambiente\\Scripts\\PPSI\\BASE\\2025-Master - Lista Orgaos SISP - Agrupamento PPSI - Protected.xlsx'

In [ ]:
df_master_indicadores = df_master_indicadores.rename(columns={'Órgão': 'Nome da Origem.1'})

In [ ]:
df_master_indicadores = df_master_indicadores.rename(columns={'Estrut. Básica': 'iBase'})

In [ ]:
 # Primeiro, extrai o número do ciclo como inteiro
df_master_indicadores['numero_ciclo'] = df_master_indicadores['Ciclo_Entrega'].str.extract(r'(\d+)').astype(int)

# Define o ano e semestre a partir do número do ciclo
df_master_indicadores['Ano'] = 2023 + ((df_master_indicadores['numero_ciclo'] - 1) // 2)
df_master_indicadores['Semestre'] = ((df_master_indicadores['numero_ciclo'] - 1) % 2) + 1

# Remove a coluna auxiliar se quiser
df_master_indicadores = df_master_indicadores.drop(columns='numero_ciclo')

In [ ]:
# Manter a coluna como float
df_master_indicadores['iBase'] = pd.to_numeric(df_master_indicadores['iBase'], errors='coerce')

# Exibir com vírgula (sem alterar tipo para 'object')
df_master_indicadores['iBase'] = df_master_indicadores['iBase'].apply(lambda x: f'{x:.2f}'.replace('.', ',') if pd.notnull(x) else '')


# Manter a coluna como float
df_master_indicadores['iSeg'] = pd.to_numeric(df_master_indicadores['iSeg'], errors='coerce')

# Exibir com vírgula (sem alterar tipo para 'object')
df_master_indicadores['iSeg'] = df_master_indicadores['iSeg'].apply(lambda x: f'{x:.2f}'.replace('.', ',') if pd.notnull(x) else '')

# Manter a coluna como float
df_master_indicadores['iPriv'] = pd.to_numeric(df_master_indicadores['iPriv'], errors='coerce')

# Exibir com vírgula (sem alterar tipo para 'object')
df_master_indicadores['iPriv'] = df_master_indicadores['iPriv'].apply(lambda x: f'{x:.2f}'.replace('.', ',') if pd.notnull(x) else '')

In [ ]:
df_master_indicadores.head()

In [ ]:
df_master_indicadores['Chave'] = df_master_indicadores['Ciclo_Entrega'].astype(str) + df_master_indicadores['Nome da Origem.1'].astype(str) + df_master_indicadores['Setor'].astype(str)

In [ ]:
 # Primeiro, extrai o número do ciclo como inteiro
df_master_indicadores['numero_ciclo'] = df_master_indicadores['Ciclo_Entrega'].str.extract(r'(\d+)').astype(int)

# Define o ano e semestre a partir do número do ciclo
df_master_indicadores['Ano'] = 2023 + ((df_master_indicadores['numero_ciclo'] - 1) // 2)
df_master_indicadores['Semestre'] = ((df_master_indicadores['numero_ciclo'] - 1) % 2) + 1

# Remove a coluna auxiliar se quiser
df_master_indicadores = df_master_indicadores.drop(columns='numero_ciclo')

In [ ]:
df_master_indicadores.head()

In [ ]:
df_master_indicadores.info()

In [ ]:
df_master_indicadores = df_master_indicadores[df_master_indicadores['Nome da Origem.1'].notna()]
df_master_indicadores['Nome da Origem.1'] = df_master_indicadores['Nome da Origem.1'].str.strip()
df_master_indicadores['Órgão-Sigla'] = df_master_indicadores['Órgão-Sigla'].str.strip()

In [ ]:
df_master_indicadores.head()

In [ ]:
df_master_indicadores.head()

In [ ]:
# Substitui "C1" por "Ciclo 1", "C2" por "Ciclo 2", etc.
df_master_indicadores['Ciclo_Entrega'] = df_master_indicadores['Ciclo_Entrega'].str.extract(r'(\d+)')[0].astype(int).apply(lambda x: f'Ciclo {x}')

In [ ]:
df_master_indicadores.head()

In [ ]:
# Salva em Excel
df_master_indicadores.to_csv(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\FONTE_PAINEL\MASTER_INDICADORES.csv", index=False)

### MASTER_OUTROS_CONSOLIDADO

In [ ]:
# Mapeamento: nome da aba → (índices das colunas a ler, nomes novos)

colunas_por_aba = {
    'Historico-MASTER-C1': 
                ([0, 1, 2, 3, 4, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 
                  37], 
                ['Resp. Titular','Categoria','Órgão','Entregou PPSI?','Data 1ª Entrega','Data Últ. Atualiz.','Infraestrutura Principal de TI',
                 'Situação de TI','Provedor','Tipo','Órgão.1','Órgão-Sigla','Órgão-Nome','Setor','UF','Município','Região','Gestor de TI SISP',
                 'e-mail RESP. Sisp','Tel. Resp. Sisp','CHEFE AECI/AI & SECRETÁRIO CONTROLE INTERNO','e-mail AECI/AI/SCI','Tel. AECI/AI/SCI',
                 'encarregado','e-mail encarregado','tel encarregado','Gestor de SI','e-mail SI','Tel SI','Contatos Opcionais (Cargo/Área)',
                 'Nome (Contatos Opcionais)','E-mail (Contatos Opcionais)','Tel (Contatos Opcionais)','STATUS Estrutura PPSI','Fonte AECI:']),
    'Historico-MASTER-C2':  
                ([ 0, 1, 2, 3, 4, 5, 6, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
                   40, 41, 42, 43],
                ['Resp. Titular','Site Atualiz. Órgão Sisp','Categoria','Órgão','Entregou PPSI?','Data 1ª Entrega','Data Últ. Atualiz.',
                 'Infraestrutura Principal de TI','Situação de TI','Provedor','Tipo','Órgão.1','Ind. Ministra','Ind. Ministra Atualizado?',
                 'Órgão-Sigla','Órgão-Nome','Setor','UF','Município','Região','Gestor de TI SISP','e-mail RESP. Sisp','Tel. Resp. Sisp',
                 'CHEFE AECI/AI & SECRETÁRIO CONTROLE INTERNO','e-mail AECI/AI/SCI','Tel. AECI/AI/SCI','encarregado','e-mail encarregado',
                 'tel encarregado','Gestor de SI','e-mail SI','Tel SI','Contatos Opcionais (Cargo/Área)','Nome (Contatos Opcionais)',
                 'E-mail (Contatos Opcionais)','Tel (Contatos Opcionais)','STATUS Estrutura PPSI','Fonte AECI:']),
    'Historico-MASTER-C3': 
                ([ 0, 1, 2, 3, 4, 5, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,
                   43, 44, 45, 46],
                ['Resp. Titular','Site Atualiz. Órgão Sisp','Categoria','Órgão','Entregou PPSI?','Data 1ª Entrega','Data Últ. Atualiz.',
                 'Infraestrutura Principal de TI','Situação de TI','Provedor','Tipo','Órgão.1','Ind. Ministra','Ind. Ministra Atualizado?',
                 'Órgão-Sigla','Órgão-Nome','Setor','UF','Município','Região','Gestor de TI SISP','e-mail RESP. Sisp','Tel. Resp. Sisp',
                 'CHEFE AECI/AI & SECRETÁRIO CONTROLE INTERNO','e-mail AECI/AI/SCI','Tel. AECI/AI/SCI','encarregado','e-mail encarregado',
                 'tel encarregado','Gestor de SI','e-mail SI','Tel SI','Contatos Opcionais (Cargo/Área)','Nome (Contatos Opcionais)',
                 'E-mail (Contatos Opcionais)','Tel (Contatos Opcionais)','STATUS Estrutura PPSI','Fonte AECI:']),
    'Atual-MASTER-C4': 
                ([ 0, 1, 2, 3, 4, 5, 6, 7, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
                   46, 47, 48, 49, 50],
                ['Resp. Titular','Resp. Substituto','Site Atualiz. Órgão Sisp','Categoria','Órgão','Entregou PPSI?','Data 1ª Entrega',
                 'Data Últ. Atualiz.','Infraestrutura Principal de TI','Situação de TI','Provedor','Tipo','Órgão.1','Ind. Ministra',
                 'Ind. Ministra Atualizado?','Órgão-Sigla','Órgão-Nome','Setor','UF','Município','Região','Gestor de TI SISP','e-mail RESP. Sisp',
                 'Tel. Resp. Sisp','CHEFE AECI/AI & SECRETÁRIO CONTROLE INTERNO','e-mail AECI/AI/SCI','Tel. AECI/AI/SCI','encarregado',
                 'e-mail encarregado','tel encarregado','Gestor de SI','e-mail SI','Tel SI','Contatos Opcionais (Cargo/Área)',
                 'Nome (Contatos Opcionais)','E-mail (Contatos Opcionais)','Tel (Contatos Opcionais)','STATUS Estrutura PPSI','Fonte AECI:'])
}
    
# Mapeamento: nome da aba → valor da coluna 'CICLO'
ciclo_por_aba = {
    'Historico-MASTER-C1': 'C1',
    'Historico-MASTER-C2': 'C2',
    'Historico-MASTER-C3': 'C3',
    'Atual-MASTER-C4': 'C4'    
}

# Passo 1: Obter todos os nomes de colunas únicos em todas as abas
todos_nomes_colunas = set()
for _, (_, nomes_colunas) in colunas_por_aba.items():
    todos_nomes_colunas.update(nomes_colunas)

todos_nomes_colunas = list(todos_nomes_colunas)

# Passo 2: Função para ler as abas com os mapeamentos e preencher colunas faltantes
def ler_abas_padronizadas(caminho_arquivo, colunas_por_aba):
    dados_abas = {}
    for aba, (indices, nomes_colunas) in colunas_por_aba.items():
        df = pd.read_excel(caminho_arquivo, sheet_name=aba, usecols=indices)
        df.columns = nomes_colunas

        # Adiciona colunas faltantes com NaN
        for col in todos_nomes_colunas:
            if col not in df.columns:
                df[col] = None
        
        # Reordena para manter a mesma ordem de colunas
        df = df[todos_nomes_colunas]
        dados_abas[aba] = df

    return dados_abas

# Mapeamento: nome da aba → colunas a excluir
colunas_excluir_por_aba = {
    'Historico-MASTER-C1': ['Estrut. Básica', 'iSeg', 'iPriv'],
    'Historico-MASTER-C2': ['(C2) Estrut. Básica', '(C2) iSeg', '(C2) iPriv', '(C1) Estrut. Básica', '(C1) iSeg', '(C1) iPriv'],
    'Historico-MASTER-C3': ['(C3) Estrut. Básica', '(C3) iSeg', '(C3) iPriv','(C2) Estrut. Básica', '(C2) iSeg', '(C2) iPriv', 
                            '(C1) Estrut. Básica', '(C1) iSeg', '(C1) iPriv'],
    'Atual-MASTER-C4':     ['(C4) Estrut. Básica', '(C4) iSeg', '(C4) iPriv','(C3) Estrut. Básica', '(C3) iSeg', '(C3) iPriv',
                            '(C2) Estrut. Básica', '(C2) iSeg', '(C2) iPriv', '(C1) Estrut. Básica', '(C1) iSeg', '(C1) iPriv']
}

tabelas = []

for nome_aba, (indices, nomes_colunas) in colunas_por_aba.items():
    # Lê as colunas pelos índices
    df = pd.read_excel(arquivo, sheet_name=nome_aba, usecols=indices, header=0)

    # Remove colunas indesejadas com base nos nomes originais
    colunas_excluir = colunas_excluir_por_aba.get(nome_aba, [])
    df.drop(columns=colunas_excluir, inplace=True, errors='ignore')  # ignora se alguma não existir

    # Renomeia as colunas restantes conforme os nomes desejados
    df.columns = [novo for antigo, novo in zip(df.columns, nomes_colunas) if antigo not in colunas_excluir]

    # Adiciona coluna do ciclo
    df['Ciclo_Entrega'] = ciclo_por_aba[nome_aba]

    tabelas.append(df)

# Junta todas as tabelas em um único DataFrame
df_master_outros = pd.concat(tabelas, ignore_index=True)

# Remove linhas totalmente vazias (caso existam)
df_master_outros.dropna(how='all', inplace=True)

# Remove duplicatas, se necessário (baseado nas colunas principais)
df_master_outros.drop_duplicates(inplace=True)

# Reorganiza colunas para trazer 'CICLO' para frente, se quiser
cols = ['Ciclo_Entrega'] + [col for col in df_master_outros.columns if col != 'Ciclo_Entrega']
df_master_outros = df_master_outros[cols]

# Exibe um resumo
print(f'Total de registros combinados: {len(df_master_outros)}')


In [ ]:
df_master_outros.info()

In [ ]:
df_master_outros = df_master_outros.rename(columns={'Órgão': 'Nome da Origem.1'})

In [ ]:
df_master_outros['Chave'] = df_master_outros['Ciclo_Entrega'].astype(str) + df_master_outros['Nome da Origem.1'].astype(str) + df_master_outros['Setor'].astype(str)

In [ ]:
 # Primeiro, extrai o número do ciclo como inteiro
df_master_outros['numero_ciclo'] = df_master_outros['Ciclo_Entrega'].str.extract(r'(\d+)').astype(int)

# Define o ano e semestre a partir do número do ciclo
df_master_outros['Ano'] = 2023 + ((df_master_outros['numero_ciclo'] - 1) // 2)
df_master_outros['Semestre'] = ((df_master_outros['numero_ciclo'] - 1) % 2) + 1

# Remove a coluna auxiliar se quiser
df_master_outros = df_master_outros.drop(columns='numero_ciclo')

In [ ]:
df_master_outros = df_master_outros[df_master_outros['Nome da Origem.1'].notna()]
df_master_outros['Nome da Origem.1'] = df_master_outros['Nome da Origem.1'].str.strip()
df_master_outros['Órgão-Sigla'] = df_master_outros['Órgão-Sigla'].str.strip()

In [ ]:
df_master_outros.head()

In [ ]:
df_master_outros.info()

In [ ]:
# Substitui "C1" por "Ciclo 1", "C2" por "Ciclo 2", etc.
df_master_outros['Ciclo_Entrega'] = df_master_outros['Ciclo_Entrega'].str.extract(r'(\d+)')[0].astype(int).apply(lambda x: f'Ciclo {x}')

In [ ]:
# Salva em Excel
df_master_outros.to_csv(r"C:\Users\denis\OneDrive\Documentos\python\meuambiente\Scripts\PPSI\FONTE_PAINEL\MASTER_OUTROS.csv", index=False)

In [ ]:
dh_termino = datetime.today()
duracao = dh_termino - dh_inicio
print(dh_inicio)
print(dh_termino)
print(duracao)